# [最高分](https://www.kaggle.com/code/honganzhu/cmi-piu-competition)

In [1]:
%pip install numpy pandas colorama catboost dask[dataframe]

Note: you may need to restart the kernel to use updated packages.


## config

In [2]:
from enum import Enum
import os
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim
from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import (
    VotingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
import copy

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None


class EXEC_ENV_ENUM(Enum):
    COLAB = 1
    KAGGLE = 2
    LOCAL = 3


"""
丟到 kaggle 要改的地方
"""
# EXEC_ENV = EXEC_ENV_ENUM.COLAB
EXEC_ENV = EXEC_ENV_ENUM.KAGGLE
# EXEC_ENV = EXEC_ENV_ENUM.LOCAL

"""
Constant
"""
train_featuresCols = (
    [  # 三次 submission train 的 feature columns (第二跟第三次的 column 一樣)
        "Basic_Demos-Age",
        "Basic_Demos-Sex",
        "CGAS-CGAS_Score",
        "Physical-BMI",
        "Physical-Height",
        "Physical-Weight",
        "Physical-Waist_Circumference",
        "Physical-Diastolic_BP",
        "Physical-HeartRate",
        "Physical-Systolic_BP",
        "Fitness_Endurance-Max_Stage",
        "Fitness_Endurance-Time_Mins",
        "Fitness_Endurance-Time_Sec",
        "FGC-FGC_CU",
        "FGC-FGC_CU_Zone",
        "FGC-FGC_GSND",
        "FGC-FGC_GSND_Zone",
        "FGC-FGC_GSD",
        "FGC-FGC_GSD_Zone",
        "FGC-FGC_PU",
        "FGC-FGC_PU_Zone",
        "FGC-FGC_SRL",
        "FGC-FGC_SRL_Zone",
        "FGC-FGC_SRR",
        "FGC-FGC_SRR_Zone",
        "FGC-FGC_TL",
        "FGC-FGC_TL_Zone",
        "BIA-BIA_Activity_Level_num",
        "BIA-BIA_BMC",
        "BIA-BIA_BMI",
        "BIA-BIA_BMR",
        "BIA-BIA_DEE",
        "BIA-BIA_ECW",
        "BIA-BIA_FFM",
        "BIA-BIA_FFMI",
        "BIA-BIA_FMI",
        "BIA-BIA_Fat",
        "BIA-BIA_Frame_num",
        "BIA-BIA_ICW",
        "BIA-BIA_LDM",
        "BIA-BIA_LST",
        "BIA-BIA_SMM",
        "BIA-BIA_TBW",
        "PAQ_A-PAQ_A_Total",
        "PAQ_C-PAQ_C_Total",
        "SDS-SDS_Total_Raw",
        "SDS-SDS_Total_T",
        "PreInt_EduHx-computerinternet_hoursday",
        "sii",
        "BMI_Age",
        "Internet_Hours_Age",
        "BMI_Internet_Hours",
        "BFP_BMI",
        "FFMI_BFP",
        "FMI_BFP",
        "LST_TBW",
        "BFP_BMR",
        "BFP_DEE",
        "BMR_Weight",
        "DEE_Weight",
        "SMM_Height",
        "Muscle_to_Fat",
        "Hydration_Status",
        "ICW_TBW",
    ],
    [
        "Basic_Demos-Enroll_Season",
        "Basic_Demos-Age",
        "Basic_Demos-Sex",
        "CGAS-Season",
        "CGAS-CGAS_Score",
        "Physical-Season",
        "Physical-BMI",
        "Physical-Height",
        "Physical-Weight",
        "Physical-Waist_Circumference",
        "Physical-Diastolic_BP",
        "Physical-HeartRate",
        "Physical-Systolic_BP",
        "Fitness_Endurance-Season",
        "Fitness_Endurance-Max_Stage",
        "Fitness_Endurance-Time_Mins",
        "Fitness_Endurance-Time_Sec",
        "FGC-Season",
        "FGC-FGC_CU",
        "FGC-FGC_CU_Zone",
        "FGC-FGC_GSND",
        "FGC-FGC_GSND_Zone",
        "FGC-FGC_GSD",
        "FGC-FGC_GSD_Zone",
        "FGC-FGC_PU",
        "FGC-FGC_PU_Zone",
        "FGC-FGC_SRL",
        "FGC-FGC_SRL_Zone",
        "FGC-FGC_SRR",
        "FGC-FGC_SRR_Zone",
        "FGC-FGC_TL",
        "FGC-FGC_TL_Zone",
        "BIA-Season",
        "BIA-BIA_Activity_Level_num",
        "BIA-BIA_BMC",
        "BIA-BIA_BMI",
        "BIA-BIA_BMR",
        "BIA-BIA_DEE",
        "BIA-BIA_ECW",
        "BIA-BIA_FFM",
        "BIA-BIA_FFMI",
        "BIA-BIA_FMI",
        "BIA-BIA_Fat",
        "BIA-BIA_Frame_num",
        "BIA-BIA_ICW",
        "BIA-BIA_LDM",
        "BIA-BIA_LST",
        "BIA-BIA_SMM",
        "BIA-BIA_TBW",
        "PAQ_A-Season",
        "PAQ_A-PAQ_A_Total",
        "PAQ_C-Season",
        "PAQ_C-PAQ_C_Total",
        "SDS-Season",
        "SDS-SDS_Total_Raw",
        "SDS-SDS_Total_T",
        "PreInt_EduHx-Season",
        "PreInt_EduHx-computerinternet_hoursday",
        "sii",
    ],
)
test_featuresCols = [
        "Basic_Demos-Age",
        "Basic_Demos-Sex",
        "CGAS-CGAS_Score",
        "Physical-BMI",
        "Physical-Height",
        "Physical-Weight",
        "Physical-Waist_Circumference",
        "Physical-Diastolic_BP",
        "Physical-HeartRate",
        "Physical-Systolic_BP",
        "Fitness_Endurance-Max_Stage",
        "Fitness_Endurance-Time_Mins",
        "Fitness_Endurance-Time_Sec",
        "FGC-FGC_CU",
        "FGC-FGC_CU_Zone",
        "FGC-FGC_GSND",
        "FGC-FGC_GSND_Zone",
        "FGC-FGC_GSD",
        "FGC-FGC_GSD_Zone",
        "FGC-FGC_PU",
        "FGC-FGC_PU_Zone",
        "FGC-FGC_SRL",
        "FGC-FGC_SRL_Zone",
        "FGC-FGC_SRR",
        "FGC-FGC_SRR_Zone",
        "FGC-FGC_TL",
        "FGC-FGC_TL_Zone",
        "BIA-BIA_Activity_Level_num",
        "BIA-BIA_BMC",
        "BIA-BIA_BMI",
        "BIA-BIA_BMR",
        "BIA-BIA_DEE",
        "BIA-BIA_ECW",
        "BIA-BIA_FFM",
        "BIA-BIA_FFMI",
        "BIA-BIA_FMI",
        "BIA-BIA_Fat",
        "BIA-BIA_Frame_num",
        "BIA-BIA_ICW",
        "BIA-BIA_LDM",
        "BIA-BIA_LST",
        "BIA-BIA_SMM",
        "BIA-BIA_TBW",
        "PAQ_A-PAQ_A_Total",
        "PAQ_C-PAQ_C_Total",
        "SDS-SDS_Total_Raw",
        "SDS-SDS_Total_T",
        "PreInt_EduHx-computerinternet_hoursday",
        "BMI_Age",
        "Internet_Hours_Age",
        "BMI_Internet_Hours",
        "BFP_BMI",
        "FFMI_BFP",
        "FMI_BFP",
        "LST_TBW",
        "BFP_BMR",
        "BFP_DEE",
        "BMR_Weight",
        "DEE_Weight",
        "SMM_Height",
        "Muscle_to_Fat",
        "Hydration_Status",
        "ICW_TBW",
    ]
season_cols = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season',
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season',
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

SEED = 42
n_splits = 5  # Statified K-Fold 的 fold 數
LGBM_Params = {
    "learning_rate": 0.046,
    "max_depth": 12,
    "num_leaves": 478,
    "min_data_in_leaf": 13,
    "feature_fraction": 0.893,
    "bagging_fraction": 0.784,
    "bagging_freq": 4,
    "lambda_l1": 10,  # Increased from 6.59
    "lambda_l2": 0.01,  # Increased from 2.68e-06
    # 'device': 'gpu'
}
XGB_Params = {
    "learning_rate": 0.05,
    "max_depth": 6,
    "n_estimators": 200,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 1,  # Increased from 0.1
    "reg_lambda": 5,  # Increased from 1
    "random_state": SEED,
    # 'tree_method': 'gpu_hist',
}
CatBoost_Params_original = {
    "learning_rate": 0.05,
    "depth": 6,
    "iterations": 200,
    "random_seed": SEED,
    "verbose": 0,
    "l2_leaf_reg": 10,  # Increase this value
    # 'task_type': 'GPU'
}
if EXEC_ENV != EXEC_ENV_ENUM.LOCAL:  # Enable GPU
    LGBM_Params['device'] = 'gpu'
    XGB_Params['tree_method'] = 'gpu_hist'
    CatBoost_Params_original['task_type'] = 'GPU'

"""
計算 input_path
"""
input_path = ""
if EXEC_ENV == EXEC_ENV_ENUM.COLAB:
    from google.colab import drive
    drive.mount("/content/gdrive")
    input_path = (
        "/content/gdrive/MyDrive/DM/child-mind-institute-problematic-internet-use"
    )
elif EXEC_ENV == EXEC_ENV_ENUM.KAGGLE:
    input_path = "/kaggle/input/child-mind-institute-problematic-internet-use"
    os.chdir("/kaggle/working")
else:
    input_path = "child-mind-institute-problematic-internet-use"

## Class & Function

In [3]:
class Preprocessing:
    def __init__(self, train_csv_input_path,
                 test_csv_input_path, 
                 train_actigraphy_input_path, 
                 test_actigraphy_input_path):
        self.__train = pd.read_csv(train_csv_input_path)
        self.__test = pd.read_csv(test_csv_input_path)
        self.__train_ts = self.__load_time_series(train_actigraphy_input_path)
        self.__test_ts = self.__load_time_series(test_actigraphy_input_path)
        # self.__train_act_data = self.__process_pca_on_parquet(train_actigraphy_input_path)
        # self.__test_act_data = self.__process_pca_on_parquet(test_actigraphy_input_path)
        
        # 儲存 id
        self.train_id = self.__train["id"]
        self.test_id = self.__test["id"]
        self.train_ts_id = self.__train_ts["id"]
        self.test_ts_id = self.__test_ts["id"]

    def __load_time_series(self, dirname) -> pd.DataFrame:
        """
        回傳 actigraphy 的 dataframe (col: 依序所有 actigraphy column 的 count/mean/std/min/23%/50%/75%/max)，並有一個 col "id"
        """

        def process_file(filename, dirname):
            """
            回傳 (actigraphy 依序所有 column 的 count/mean/std/min/23%/50%/75%/max 的一維陣列, id)
            """

            df = pd.read_parquet(os.path.join(dirname, filename, "part-0.parquet"))
            df.drop("step", axis=1, inplace=True)
            return df.describe().values.reshape(-1), filename.split("=")[1]

        ids = os.listdir(dirname)

        with ThreadPoolExecutor() as executor:
            results = list(
                tqdm(
                    executor.map(lambda fname: process_file(fname, dirname), ids),
                    total=len(ids),
                )
            )

        stats, indexes = zip(*results)

        df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
        df["id"] = indexes
        return df
    
    def __change_season_col_dtype_to_category(self, df, cat_c):
        for c in cat_c:
            df[c] = df[c].fillna('Missing')  # 把 NaN (missing value) 改成字串 "Missing"
            df[c] = df[c].astype('category')  # 把這些 column 的 dtype 從 object 改成 category (但型別還是 string) (改不改應該對結果沒什麼影響)
        return df
    def __process_pca_on_parquet(self, dir_path, n_components=5):
        # 取得 parquet 文件的列表
        train_par = os.listdir(dir_path)
    
        # 生成新的欄位名稱 (PCA_1, PCA_2, ..., PCA_n_components)
        new_columns = [f'PCA_{i}' for i in range(1, n_components + 1)]
        new_columns = ['id'] + new_columns
    
        # 創建最終的 DataFrame，用於保存每個文件處理後的結果
        master_df = pd.DataFrame(columns=new_columns)
    
        # 定義檔案目錄
        filedir = dir_path if dir_path.endswith('/') else dir_path + '/'
    
        # 遍歷每一個 parquet 文件資料夾
        for _dir in train_par:
            file_name = filedir + _dir + '/part-0.parquet'
            # 取得 id (檔案名稱中 split 出來的值)
            id = _dir.split('=')[1]
    
            # 使用 polars 讀取 parquet 文件
            act_data = pl.read_parquet(file_name)
    
            # 標準化數據
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(act_data)
    
            # 應用 PCA，將數據壓縮到指定維度
            pca = PCA(n_components=n_components)
            X_pca = pca.fit_transform(X_scaled)
    
            # 將壓縮後的數據轉換成 1x10 向量 (取平均)
            pca_vector = X_pca.mean(axis=0).reshape(1, -1)
            pca_vector = pca_vector.flatten().astype(object)
    
            # 插入 id 到向量的第一個位置
            flattened = np.insert(pca_vector, 0, id)
    
            # 創建一個臨時 DataFrame 保存當前結果
            temp_df = pd.DataFrame([flattened], columns=new_columns)
    
            # 將臨時 DataFrame 合併到最終的 master_df
            master_df = pd.concat([master_df, temp_df], ignore_index=True)
    
        # 返回最終的處理後 DataFrame
        return master_df

    def type1(self):
        """Submission 1"""
        
        def feature_engineering(df):
            season_cols = [col for col in df.columns if "Season" in col]
            df = df.drop(season_cols, axis=1)
            df["BMI_Age"] = df["Physical-BMI"] * df["Basic_Demos-Age"]
            df["Internet_Hours_Age"] = (
                df["PreInt_EduHx-computerinternet_hoursday"] * df["Basic_Demos-Age"]
            )
            df["BMI_Internet_Hours"] = (
                df["Physical-BMI"] * df["PreInt_EduHx-computerinternet_hoursday"]
            )
            df["BFP_BMI"] = df["BIA-BIA_Fat"] / df["BIA-BIA_BMI"]
            df["FFMI_BFP"] = df["BIA-BIA_FFMI"] / df["BIA-BIA_Fat"]
            df["FMI_BFP"] = df["BIA-BIA_FMI"] / df["BIA-BIA_Fat"]
            df["LST_TBW"] = df["BIA-BIA_LST"] / df["BIA-BIA_TBW"]
            df["BFP_BMR"] = df["BIA-BIA_Fat"] * df["BIA-BIA_BMR"]
            df["BFP_DEE"] = df["BIA-BIA_Fat"] * df["BIA-BIA_DEE"]
            df["BMR_Weight"] = df["BIA-BIA_BMR"] / df["Physical-Weight"]
            df["DEE_Weight"] = df["BIA-BIA_DEE"] / df["Physical-Weight"]
            df["SMM_Height"] = df["BIA-BIA_SMM"] / df["Physical-Height"]
            df["Muscle_to_Fat"] = df["BIA-BIA_SMM"] / df["BIA-BIA_FMI"]
            df["Hydration_Status"] = df["BIA-BIA_TBW"] / df["Physical-Weight"]
            df["ICW_TBW"] = df["BIA-BIA_ICW"] / df["BIA-BIA_TBW"]

            return df

        def revise_word_data(df):
            season_columns = df.filter(like="Seas").columns
            # 定義季節的映射關係
            season_mapping = {
                "Spring": 1,
                "Summer": 2,
                "Fall": 3,
                "Winter": 4
            }

            # 將季節欄位轉換為數值
            for col in season_columns:
                df[col] = df[col].map(season_mapping)

            # 返回處理後的 DataFrame
            return df            

        def merge_file(org, act):
            file1 = org
            file2 = act
            merged_file = pd.merge(file1, file2, on='id', how = 'left')

            return merged_file
        
        def impute(dataframe):
            df = dataframe

            df_numeric = df.select_dtypes(include=['float64', 'int64'])  # 只選擇數值型欄位

            # 使用 SimpleImputer 進行填補
            imputer = KNNImputer(n_neighbors=5)  # 可以調整 n_neighbors 來設定最近鄰居的數量
            df_numeric_imputed = pd.DataFrame(imputer.fit_transform(df_numeric), columns=df_numeric.columns)
            df_numeric_imputed["sii"] = df_numeric_imputed["sii"].round().astype(int)

            # 替換原 DataFrame 中的數值型欄位
            df[df_numeric.columns] = df_numeric_imputed

            # 返回處理後的 DataFrame
            return df

        def drop_features(train, test):
            features_not_in_test = []
            for f in train.columns:
                if f not in test.columns:
                    features_not_in_test.append(f)

            print(features_not_in_test)

            features_not_in_test.pop()
            features_not_in_test.pop(0)
            train_revised = train.drop(columns=features_not_in_test)

            return train_revised

        def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
            # 將 dataframe 轉成 z-score
            scaler = StandardScaler()
            df_scaled = scaler.fit_transform(df)

            data_tensor = torch.FloatTensor(df_scaled)

            input_dim = data_tensor.shape[1]
            autoencoder = AutoEncoder(input_dim, encoding_dim)

            criterion = nn.MSELoss()
            optimizer = optim.Adam(autoencoder.parameters())

            for epoch in range(epochs):
                for i in range(0, len(data_tensor), batch_size):
                    batch = data_tensor[i : i + batch_size]
                    optimizer.zero_grad()
                    reconstructed = autoencoder(batch)
                    loss = criterion(reconstructed, batch)
                    loss.backward()
                    optimizer.step()

                if (epoch + 1) % 10 == 0:
                    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]")

            with torch.no_grad():
                encoded_data = autoencoder.encoder(data_tensor).numpy()

            df_encoded = pd.DataFrame(
                encoded_data, columns=[f"Enc_{i + 1}" for i in range(encoded_data.shape[1])]
            )

            return df_encoded
        def perform_pca(df, n_components):
            """
            Perform PCA on the given DataFrame and return the reduced DataFrame.

            Parameters:
            - df: DataFrame to process
            - n_components: Number of principal components to keep

            Returns:
            - A DataFrame with reduced dimensions
            """
            # Z-score normalization
            scaler = StandardScaler()
            df_scaled = scaler.fit_transform(df)

            # Apply PCA
            pca = PCA(n_components=n_components)
            reduced_data = pca.fit_transform(df_scaled)

            # Convert to DataFrame
            df_reduced = pd.DataFrame(
                reduced_data, columns=[f"PCA_{i + 1}" for i in range(n_components)]
            )
            return df_reduced
        
        def perform_anova(df, df_test):
            labels = df['sii']
            df = df.drop(['sii'], axis=1)

            X = df.iloc[:, :-1]
            y = df.iloc[:, -1]

            imputer = KNNImputer(n_neighbors=5)
            X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)  

            selector = SelectKBest(score_func=f_classif, k=20)
            selector.fit(X_imputed, y)

            selected_features = X.columns[selector.get_support()]

            new_df = df[selected_features]
            df_test = df_test[selected_features]
            new_df['sii'] = labels

            return new_df, df_test
        
        def perform_rf(df, df_test):
            labels = df['sii']
            df = df.drop(['sii'], axis=1)

            # 特徵矩陣與標籤
            X = df
            y = labels

            # 缺失值補全
            imputer = KNNImputer(n_neighbors=5)
            X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

            # 使用 Random Forest 訓練模型
            rf = RandomForestClassifier(n_estimators=100, random_state=42)
            rf.fit(X_imputed, y)

            # 根據特徵重要性選擇前 20 個特徵
            feature_importances = rf.feature_importances_
            important_indices = feature_importances.argsort()[-20:][::-1]
            selected_features = X.columns[important_indices]

            # 篩選新特徵
            new_df = X_imputed[selected_features]
            df_test = df_test[selected_features]
            new_df['sii'] = labels

            return new_df, df_test
        
        def perform_lasso(df, df_test):
            labels = df['sii']
            df = df.drop(['sii'], axis=1)

            # 特徵矩陣與標籤
            X = df
            y = labels

            # 缺失值補全
            imputer = KNNImputer(n_neighbors=5)
            X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

            # 標準化數據
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_imputed)

            # 使用 Lasso 訓練模型
            lasso = Lasso(alpha=0.01, random_state=42)  # 可以根據需求調整 alpha
            lasso.fit(X_scaled, y)

            # 選擇非零權重的特徵
            selected_features = X.columns[lasso.coef_ != 0]

            # 篩選新特徵
            new_df = X_imputed[selected_features]
            df_test = df_test[selected_features]
            new_df['sii'] = labels

            return new_df, df_test            

        train = copy.deepcopy(self.__train)
        test = copy.deepcopy(self.__test)

        # Load in actigraphy data
        train_ts = copy.deepcopy(self.__train_ts)
        test_ts = copy.deepcopy(self.__test_ts)

        train_ts = train_ts.drop("id", axis=1)
        test_ts = test_ts.drop("id", axis=1)

        # 對 actigraphy data 用 autoencoder 做 encode
        train_ts = perform_autoencoder(
            train_ts, encoding_dim=60, epochs=100, batch_size=32
        )
        test_ts = perform_autoencoder(
            test_ts, encoding_dim=60, epochs=100, batch_size=32
        )

        # train_ts = perform_pca(train_ts, n_components=2)
        # test_ts = perform_pca(test_ts, n_components=2)


        encode_cols_name = train_ts.columns.tolist()  # Enc_1 ~ Enc_<encoding_dim>

        train_ts["id"] = self.train_ts_id
        test_ts["id"] = self.test_ts_id

        # Merge original data and actigraphy
        merged_train = merge_file(train, train_ts)
        merged_test = merge_file(test, test_ts)

        # Use KNN imputer to impute training dataset
        train_imputed = impute(merged_train)

        # Revise word data (Season)
        train_imputed = revise_word_data(train_imputed)
        merged_test = revise_word_data(merged_test)

        train_imputed = drop_features(train_imputed, merged_test)

        # Seperate into two training dataset (有做測驗 & 沒做測驗)
        train_PCIAT = train_imputed[train_imputed['PCIAT-Season'].notna()]
        train_No_PCIAT = train_imputed[train_imputed['PCIAT-Season'].isna()]

        train_PCIAT = train_PCIAT.drop("PCIAT-Season", axis=1)
        train_No_PCIAT = train_No_PCIAT.drop("PCIAT-Season", axis=1)

        # Feature engineering
        train_PCIAT = feature_engineering(train_PCIAT)
        train_No_PCIAT = feature_engineering(train_No_PCIAT)
        merged_test = feature_engineering(merged_test)
        train_imputed = feature_engineering(train_imputed)

        # 移除 id
        train_PCIAT = train_PCIAT.drop("id", axis=1)
        train_NO_PCIAT = train_No_PCIAT.drop("id", axis=1)
        merged_test = merged_test.drop("id", axis=1)

        train_imputed = train_imputed.drop("id", axis=1)

        if np.any(np.isinf(train_PCIAT)):
            train_PCIAT = train_PCIAT.replace([np.inf, -np.inf], np.nan)
        if np.any(np.isinf(train_NO_PCIAT)):
            train_PCIAT = train_NO_PCIAT.replace([np.inf, -np.inf], np.nan)
        if np.any(np.isinf(train_imputed)):
            train_imputed = train_imputed.replace([np.inf, -np.inf], np.nan)

        # train_anova, merged_test = perform_anova(train_imputed, merged_test)
        train_anova, merged_test = perform_lasso(train_imputed, merged_test)
        # train_anova, merged_test = perform_rf(train_imputed, merged_test)

        # return train_PCIAT, train_No_PCIAT, merged_test
        return train_anova, merged_test
    
    def type2(self):
        """Submission 2 and 3 (portion)"""

        def create_mapping(column, dataset):
            '''
            回傳 column 內的 unique value dictionary (key: 值、value: 對應的一數字)
            '''

            unique_values = dataset[column].unique()
            return {value: idx for idx, value in enumerate(unique_values)}

        train = copy.deepcopy(self.__train)
        test = copy.deepcopy(self.__test)
        train_ts = copy.deepcopy(self.__train_ts)
        test_ts = copy.deepcopy(self.__test_ts)

        # 儲存處理好 (但不做 Encode) 的 column names
        time_series_cols_name = train_ts.columns.tolist()
        time_series_cols_name.remove("id")

        # merge csv 資料跟處理好的 actigraphy data
        train = pd.merge(train, train_ts, how="left", on='id')
        test = pd.merge(test, test_ts, how="left", on='id')

        # 移除 id
        train = train.drop("id", axis=1)
        test = test.drop("id", axis=1)

        # 只取某些 column
        featuresCols = copy.deepcopy(train_featuresCols[1]) + time_series_cols_name
        train = train[featuresCols].dropna(subset='sii')

        train = self.__change_season_col_dtype_to_category(train, season_cols)
        test = self.__change_season_col_dtype_to_category(test, season_cols)

        # 將所有 season columns 的值變為 numeric
        for col in season_cols:
            mapping_train = create_mapping(col, train)
            mapping_test = create_mapping(col, test)

            train[col] = train[col].replace(mapping_train).astype(int)
            test[col] = test[col].replace(mapping_test).astype(int)

        return train, test


class AutoEncoder(nn.Module):

    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim * 3),
            nn.ReLU(),
            nn.Linear(encoding_dim * 3, encoding_dim * 2),
            nn.ReLU(),
            nn.Linear(encoding_dim * 2, encoding_dim),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim * 2),
            nn.ReLU(),
            nn.Linear(input_dim * 2, input_dim * 3),
            nn.ReLU(),
            nn.Linear(input_dim * 3, input_dim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


def TrainML(model_class, train_data, test_data):

    def threshold_Rounder(oof_non_rounded, thresholds):
        """
        1. 若 oof_non_rounded 中的值 < thresholds[0]，則將該值標記為 0。
        2. 若 oof_non_rounded 的值在 thresholds[0] 和 thresholds[1] 之間，則標記為 1。
        3. 若 oof_non_rounded 的值在 thresholds[1] 和 thresholds[2] 之間，則標記為 2。
        4. 若 oof_non_rounded 的值 >= thresholds[2]，則標記為 3。
        """

        return np.where(
            oof_non_rounded < thresholds[0],
            0,
            np.where(
                oof_non_rounded < thresholds[1],
                1,
                np.where(oof_non_rounded < thresholds[2], 2, 3),
            ),
        )

    def evaluate_predictions(thresholds, y_true, oof_non_rounded):
        rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
        return -quadratic_weighted_kappa(y_true, rounded_p)

    X = train_data.drop(["sii"], axis=1)
    y = train_data["sii"]

    """
    Statified K-Fold: K-Fold 並確保訓練集，測試集中各類別樣本的比例與原始資料集中相同

    n_splits: fold 數
    """
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    # KF = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

    train_kappas = []
    test_kappas = []

    oof_non_rounded = np.zeros(len(y), dtype=float)
    # oof_rounded = np.zeros(len(y), dtype=int)
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_indices, val_indices) in enumerate(
        tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)
    ):
        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[val_indices] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        # oof_rounded[val_indices] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(
            y_train, y_train_pred.round(0).astype(int)
        )
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_kappas.append(train_kappa)
        test_kappas.append(val_kappa)

        test_preds[:, fold] = model.predict(test_data)

        print(
            f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}"
        )
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_kappas):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_kappas):.4f}")

    """
    找到預測出來的浮點數 sii 若要轉為整數，要用 threshold 多少才能讓效果最好
    原始 threshold 為 x0，optimized 後的 threshold 為 KappaOptimizer.x
    """
    KappaOptimizer = minimize(
        evaluate_predictions,
        x0=[0.5, 1.5, 2.5],
        args=(y, oof_non_rounded),
        method="Nelder-Mead",
    )
    assert KappaOptimizer.success, "Optimization did not converge."

    oof_optimized_rounded = threshold_Rounder(oof_non_rounded, KappaOptimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_optimized_rounded)

    print(
        f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}"
    )

    """
    cross-validation 的各 fold 預測的值取 mean
    """
    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOptimizer.x)

    sample = pd.read_csv(os.path.join(input_path, "sample_submission.csv"))
    submission = pd.DataFrame({"id": sample["id"], "sii": tpTuned})

    return submission

## Preprocessing

In [4]:
preprocessing = Preprocessing(
    train_csv_input_path=os.path.join(input_path, "train.csv"),
    train_actigraphy_input_path=os.path.join(input_path, "series_train.parquet"),
    test_csv_input_path=os.path.join(input_path, "test.csv"),
    test_actigraphy_input_path=os.path.join(input_path, "series_test.parquet"))

100%|██████████| 2/2 [00:00<00:00, 10.84it/s]


## Submission 1

In [5]:
# train: With PCIAT, train2: No_PCIAT
#train, train2, test = preprocessing.type1()
train, test = preprocessing.type1()

print("Preprocessing_finished")

# Create model instances
# FIXME: 小型數據集 LGBM 容易 overfitting
Light = LGBMRegressor(
    **LGBM_Params, random_state=SEED, verbose=-1, n_estimators=300
)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params_original)

# Combine models using Voting Regressor
voting_model = VotingRegressor(
    estimators=[
        ("lightgbm", Light),
        ("xgboost", XGB_Model),
        ("catboost", CatBoost_Model),
    ]
)

# Train the ensemble model
Submission1 = TrainML(voting_model, train, test)

Submission1.to_csv("submission.csv", index=False)

Training Folds: 100%|██████████| 5/5 [00:23<00:00,  4.61s/it]

Mean Train QWK --> 0.7671
Mean Validation QWK ---> 0.4831
----> || Optimized QWK SCORE ::  0.535


## Submission 2

In [6]:
train, test = preprocessing.type2()

CatBoost_Params = copy.deepcopy(CatBoost_Params_original)
CatBoost_Params['cat_features'] = season_cols

Light = LGBMRegressor(**LGBM_Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

Submission2 = TrainML(voting_model, train, test)

Training Folds: 100%|██████████| 5/5 [00:32<00:00,  6.47s/it]

Mean Train QWK --> 0.7627
Mean Validation QWK ---> 0.3961


----> || Optimized QWK SCORE ::  0.455


## Submission 3

In [7]:
train, test = preprocessing.type2()

# Submisssion 3 另外的 preprocessing: 填 missing value 方法使用取 median
imputer = SimpleImputer(strategy='median')

ensemble = VotingRegressor(estimators=[
    ('lgb', Pipeline(steps=[('imputer', imputer), ('regressor', LGBMRegressor(random_state=SEED))])),
    ('xgb', Pipeline(steps=[('imputer', imputer), ('regressor', XGBRegressor(random_state=SEED))])),
    ('cat', Pipeline(steps=[('imputer', imputer), ('regressor', CatBoostRegressor(random_state=SEED, silent=True))])),
    ('rf', Pipeline(steps=[('imputer', imputer), ('regressor', RandomForestRegressor(random_state=SEED))])),
    ('gb', Pipeline(steps=[('imputer', imputer), ('regressor', GradientBoostingRegressor(random_state=SEED))]))
])

Submission3 = TrainML(ensemble, train, test)

Training Folds: 100%|██████████| 5/5 [02:00<00:00, 24.02s/it]

Mean Train QWK --> 0.9175
Mean Validation QWK ---> 0.3803


----> || Optimized QWK SCORE ::  0.450


## Submission 1 + 2 + 3

In [8]:
Submission1 = Submission1.sort_values(by='id').reset_index(drop=True)
Submission2 = Submission2.sort_values(by='id').reset_index(drop=True)
Submission3 = Submission3.sort_values(by='id').reset_index(drop=True)

combined = pd.DataFrame({
    'id': Submission1['id'],
    'sii_1': Submission1['sii'],
    'sii_2': Submission2['sii'],
    'sii_3': Submission3['sii']
})

def majority_vote(row):
    return row.mode()[0]  # mode: 眾數

combined['final_sii'] = combined[['sii_1', 'sii_2', 'sii_3']].apply(majority_vote, axis=1)

final_submission = combined[['id', 'final_sii']].rename(columns={'final_sii': 'sii'})

# final_submission.to_csv('submission.csv', index=False)